In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, confusion_matrix
from sklearn.preprocessing import label_binarize

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

from art.attacks.evasion import SimBA, SpatialTransformation, DeepFool, BasicIterativeMethod, FastGradientMethod, ProjectedGradientDescent
from art.estimators.classification import PyTorchClassifier

import time

/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
head = {
            "model" : '',
            "attack_model": '',
            'epsilon': '',
            'Accuracy': '',
            'Macro Precision': '',
            'Weighted Precision': '',
            'Macro Recall': '',
            'Weighted Recall': '',
            'Macro F1': '',
            'Weighted F1': '',
            # 'Macro AUC': '',
            # 'Weighted AUC': '',
            'TPR': '',
            'FNR': '',
            'TNR': '',
            'FPR': '',
        }
head = pd.DataFrame([head])
head.to_csv("/home/jovyan/Attack/attackmodel.csv", mode='a', index=False)




In [3]:
def calculate_performance_metrics(X_test, y_true, model, model_name, attack_name, eps):
    model.eval()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    all_preds = []
    all_labels = []
    probabilities = []

    num_classes = len(np.unique(y_true))
    
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
    test_loader = DataLoader(dataset=test_dataset)

    with torch.no_grad():
        
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            probabilities.extend(torch.nn.functional.softmax(outputs, dim=1).cpu().numpy())
        
        all_preds = np.array(all_preds)
        all_labels = np.array(all_labels)
        probabilities = np.array(probabilities)
        
        accuracy = accuracy_score(all_labels, all_preds)

        precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(all_labels, all_preds, average='macro')
        precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')
    
        # macro_auc = roc_auc_score(label_binarize(all_labels, classes=range(num_classes)), probabilities[:,1], average='macro')
        # weighted_auc = roc_auc_score(label_binarize(all_labels, classes=range(num_classes)), probabilities[:,1], average='weighted')

        cm = confusion_matrix(all_labels, all_preds)

        def calculate_class_metrics_macro(cm, class_index):
            TP = cm[class_index, class_index]
            FP = cm[:, class_index].sum() - TP
            FN = cm[class_index, :].sum() - TP
            TN = cm.sum() - (TP + FP + FN)
            
            TPR = TP / (TP + FN) if (TP + FN) != 0 else 0  
            TNR = TN / (TN + FP) if (TN + FP) != 0 else 0  
            FPR = FP / (FP + TN) if (FP + TN) != 0 else 0  
            FNR = FN / (FN + TP) if (FN + TP) != 0 else 0  
            
            return TPR, TNR, FPR, FNR
            
        metrics = np.array([calculate_class_metrics_macro(cm, i) for i in range(num_classes)])
        TPR_macro, TNR_macro, FPR_macro, FNR_macro = np.mean(metrics, axis=0)

        print(f"Accuracy: {accuracy}")
        
        print("\nmacro")
        print(f"Precision: {precision_macro}\nRecall: {recall_macro}\nF1 Score: {f1_macro}")
    
        print("\nweighted")
        print(f"Precision: {precision_weighted}\nRecall: {recall_weighted}\nF1 Score: {f1_weighted}")
        print()
        
        print(f"Mean FNR: {FNR_macro}\nMean TNR: {TNR_macro}\nMean FPR: {FPR_macro}\nMean TPR: {TPR_macro}")

        new_row = {
            "model" : model_name,
            "attack_model" : attack_name,
            'epsilon': eps,
            'Accuracy': accuracy,
            'Macro Precision': precision_macro,
            'Weighted Precision': precision_weighted,
            'Macro Recall': recall_macro,
            'Weighted Recall': recall_weighted,
            'Macro F1': f1_macro,
            'Weighted F1': f1_weighted,
            # 'Macro AUC': macro_auc,
            # 'Weighted AUC': weighted_auc,
            'TPR': TPR_macro,
            'FNR': FNR_macro,
            'TNR': TNR_macro,
            'FPR': FPR_macro,
        }
        new_row_df = pd.DataFrame([new_row])
        new_row_df.to_csv("/home/jovyan/A2PM/attackmodel.csv", mode='a', index=False, header=False)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

Using cuda device


In [5]:
x_test = np.load('/home/jovyan/Wustl_iiot/x_test.npy')
x_train = np.load('/home/jovyan/Wustl_iiot/x_train.npy')
x_val = np.load('/home/jovyan/Wustl_iiot/x_val.npy')
y_test = np.load('/home/jovyan/Wustl_iiot/y_test.npy')
y_train = np.load('/home/jovyan/Wustl_iiot/y_train.npy')
y_val = np.load('/home/jovyan/Wustl_iiot/y_val.npy')

In [6]:
input_shape = x_train.shape[1]
output_shape = len(np.unique(y_train))

In [7]:
class DNNModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(DNNModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, 30)
        self.fc3 = nn.Linear(30, 20)
        self.fc4 = nn.Linear(20, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

In [8]:
model = DNNModel(input_size=input_shape, output_size=output_shape).to(device)
model.load_state_dict(torch.load("/home/jovyan/Wustl_iiot/transfer_attack/dnn_pytorch.pt"))

<All keys matched successfully>

In [9]:
x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)

In [10]:
    
classifier = PyTorchClassifier(
    model=model,
    clip_values=(-5, 5),
    loss=nn.CrossEntropyLoss(),
    optimizer=optim.Adam(model.parameters(), lr=0.001),
    input_shape=(input_shape,),
    nb_classes=output_shape,
    device_type='gpu'
)

In [11]:
from art.attacks.evasion import SimBA, SpatialTransformation, DeepFool, BasicIterativeMethod, FastGradientMethod, ProjectedGradientDescent


In [12]:
epsilon_values = [0.01, 0.1, 0.2, 0.3]

# Iterate over epsilon values
for epsilon in epsilon_values:
    adv_crafter = BasicIterativeMethod(classifier, eps=epsilon, max_iter = 10, verbose=False) # init eps=0.3
    x_test_adv = adv_crafter.generate(x=x_test, y=y_test)

    filename = f'/home/jovyan/Wustl_iiot/transfer_attack/x_test_adv_BIM_eps_{epsilon}.npy'
    np.save(filename, x_test_adv)

In [13]:
for epsilon in epsilon_values:
    adv_crafter = FastGradientMethod(classifier, eps=epsilon) # init eps=0.3
    x_test_adv = adv_crafter.generate(x=x_test, y=y_test)

    filename = f'/home/jovyan/Wustl_iiot/transfer_attack/x_test_adv_FGSM_eps_{epsilon}.npy'
    np.save(filename, x_test_adv)

In [15]:
for epsilon in epsilon_values:
    adv_crafter = ProjectedGradientDescent(classifier, eps=epsilon, verbose=False, max_iter=10) # init eps=0.3
    x_test_adv = adv_crafter.generate(x=x_test, y=y_test)

    filename = f'/home/jovyan/Wustl_iiot/transfer_attack/x_test_adv_PGD_eps_{epsilon}.npy'
    np.save(filename, x_test_adv)

In [18]:
DF_eps = [0.01, 0.1, 0.2, 0.3]
for epsilon in DF_eps:
    adv_crafter = DeepFool(classifier, epsilon=epsilon, verbose=False, max_iter=10) # init eps=1e-06
    x_test_adv = adv_crafter.generate(x=x_test, y=y_test)

    filename = f'/home/jovyan/Wustl_iiot/transfer_attack/x_test_adv_DF_eps_{epsilon}.npy'
    np.save(filename, x_test_adv)


In [ ]:
epsilon_values = [0.01, 0.1, 0.2, 0.3]

# Iterate over epsilon values
for epsilon in epsilon_values:
    filename = f'/home/jovyan/Wustl_iiot/transfer_attack/x_test_adv_BIM_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'BIM', epsilon)

for epsilon in epsilon_values:
    filename = f'/home/jovyan/Wustl_iiot/transfer_attack/x_test_adv_FGSM_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'FGSM', epsilon)

for epsilon in epsilon_values:
    filename = f'/home/jovyan/Wustl_iiot/transfer_attack/x_test_adv_PGD_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'PGD', epsilon)

DF_eps = [1e-07, 1e-06, 1e-05, 1e-04]
for epsilon in DF_eps:
    filename = f'/home/jovyan/Wustl_iiot/transfer_attack/x_test_adv_DF_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'DF', epsilon)

Accuracy: 0.9999581157095409

macro
Precision: 0.9760275644517383
Recall: 0.9711603118287865
F1 Score: 0.971746830391899

weighted
Precision: 0.9999615942542808
Recall: 0.9999581157095409
F1 Score: 0.9999580733593396

Mean FNR: 0.02883968817121358
Mean TNR: 0.9999809556341834
Mean FPR: 1.9044365816625028e-05
Mean TPR: 0.9711603118287865
Accuracy: 0.9996774909634644

macro
Precision: 0.8196007792102954
Recall: 0.9389366493411517
F1 Score: 0.8659194698907322

weighted
Precision: 0.9997748264873905
Recall: 0.9996774909634644
F1 Score: 0.9997128387808352

Mean FNR: 0.06106335065884842
Mean TNR: 0.9999245569611481
Mean FPR: 7.544303885183419e-05
Mean TPR: 0.9389366493411517
Accuracy: 0.3404061938488731

macro
Precision: 0.6901771545662203
Recall: 0.7501413861932366
F1 Score: 0.5889055813084352

weighted
Precision: 0.9936338768020678
Recall: 0.3404061938488731
F1 Score: 0.48843781530459945

Mean FNR: 0.24985861380676347
Mean TNR: 0.8653903530704061
Mean FPR: 0.13460964692959376
Mean TPR: 0.7

In [ ]:
from art.attacks.evasion import ZooAttack

epsilon_values = [0, 0.01, 0.1, 0.2, 0.3]

# Iterate over epsilon values
for epsilon in epsilon_values:
    adv_crafter = ZooAttack(classifier, confidence=epsilon, max_iter = 3, batch_size=1, nb_parallel=42) # init eps=0.3
    x_test_adv = adv_crafter.generate(x=x_test, y=y_test)

    filename = f'/home/jovyan/Wustl_iiot/transfer_attack/x_test_adv_ZOO_eps_{epsilon}.npy'
    np.save(filename, x_test_adv)

In [11]:
from art.attacks.evasion import BoundaryAttack

epsilon_values = [0.01, 0.1, 0.2, 0.3]

# Iterate over epsilon values
for epsilon in epsilon_values:
    adv_crafter = BoundaryAttack(classifier, epsilon=epsilon, max_iter = 5000, batch_size=128, targeted=False, verbose=False) # init eps=0.3
    x_test_adv = adv_crafter.generate(x=x_test, y=y_test)

    filename = f'/home/jovyan/Wustl_iiot/transfer_attack/x_test_adv_BA_eps_{epsilon}.npy'
    np.save(filename, x_test_adv)

Boundary attack - iterations:   4%|▎         | 177/5000 [00:00<00:11, 424.53it/s]

Boundary attack - iterations:   4%|▍         | 219/5000 [00:00<00:11, 422.88it/s]

Boundary attack - iterations:   4%|▍         | 191/5000 [00:00<00:11, 428.37it/s]

Boundary attack - iterations:   3%|▎         | 173/5000 [00:00<00:11, 427.86it/s]

Boundary attack - iterations:   4%|▍         | 220/5000 [00:00<00:11, 422.38it/s]

Boundary attack - iterations:   4%|▍         | 213/5000 [00:00<00:11, 423.39it/s]

Boundary attack - iterations:   5%|▍         | 243/5000 [00:00<00:10, 432.67it/s]

Boundary attack - iterations:   4%|▍         | 207/5000 [00:00<00:11, 417.16it/s]

Boundary attack - iterations:   4%|▍         | 222/5000 [00:00<00:11, 422.71it/s]

Boundary attack - iterations:   4%|▍         | 221/5000 [00:00<00:11, 426.22it/s]

Boundary attack - iterations:   3%|▎         | 173/5000 [00:00<00:11, 413.62it/s]

Boundary attack: 100%|██████████| 238753/238753 [00:54<00:00, 4374.86it/s]


In [17]:
from art.attacks.evasion import AutoProjectedGradientDescent

epsilon_values = [0.01, 0.1, 0.2, 0.3]

# Iterate over epsilon values
for epsilon in epsilon_values:
    adv_crafter = AutoProjectedGradientDescent(classifier, eps=epsilon, max_iter = 10, batch_size=128, verbose = False) # init eps=0.3
    x_test_adv = adv_crafter.generate(x=x_test, y=y_test)

    filename = f'/home/jovyan/Wustl_iiot/transfer_attack/x_test_adv_AutoPGD_eps_{epsilon}.npy'
    np.save(filename, x_test_adv)

In [12]:
epsilon_values = [0.01, 0.1, 0.2, 0.3]
for epsilon in epsilon_values:
    filename = f'/home/jovyan/Wustl_iiot/transfer_attack/x_test_adv_DF_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'DF', epsilon)

for epsilon in epsilon_values:
    filename = f'/home/jovyan/Wustl_iiot/transfer_attack/x_test_adv_AutoPGD_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'AutoPGD', epsilon)

/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 6.701486473468396e-05

macro
Precision: 0.01104815321728492
Recall: 0.053859827969165074
F1 Score: 0.01831016319943083

weighted
Precision: 0.00010160209387252567
Recall: 6.701486473468396e-05
F1 Score: 3.0477766166720355e-05

Mean FNR: 0.946140172030835
Mean TNR: 0.6842493093932873
Mean FPR: 0.3157506906067126
Mean TPR: 0.053859827969165074


/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 6.701486473468396e-05

macro
Precision: 0.012041646565347635
Recall: 0.053859827969165074
F1 Score: 0.01965863204452109

weighted
Precision: 0.0001024437450261664
Recall: 6.701486473468396e-05
F1 Score: 3.194518840107166e-05

Mean FNR: 0.946140172030835
Mean TNR: 0.6839403238377363
Mean FPR: 0.3160596761622637
Mean TPR: 0.053859827969165074


/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 6.701486473468396e-05

macro
Precision: 0.013169969885609797
Recall: 0.053859827969165074
F1 Score: 0.02114158163841404

weighted
Precision: 0.00010345726723808033
Recall: 6.701486473468396e-05
F1 Score: 3.3558271549052856e-05

Mean FNR: 0.946140172030835
Mean TNR: 0.6836623365182317
Mean FPR: 0.3163376634817682
Mean TPR: 0.053859827969165074


/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 6.701486473468396e-05

macro
Precision: 0.013682941814196655
Recall: 0.053859827969165074
F1 Score: 0.021799387673622515

weighted
Precision: 0.00010390859975546363
Recall: 6.701486473468396e-05
F1 Score: 3.4273624682593036e-05

Mean FNR: 0.946140172030835
Mean TNR: 0.6835232746637144
Mean FPR: 0.3164767253362856
Mean TPR: 0.053859827969165074
Accuracy: 0.9989654580256583

macro
Precision: 0.7542382280104623
Recall: 0.8708817067630594
F1 Score: 0.7799222175915298

weighted
Precision: 0.9993421012133378
Recall: 0.9989654580256583
F1 Score: 0.999091795141612

Mean FNR: 0.12911829323694057
Mean TNR: 0.9997658794773546
Mean FPR: 0.00023412052264539174
Mean TPR: 0.8708817067630594
Accuracy: 0.0024669847080455534

macro
Precision: 0.007136366390016356
Recall: 0.0005321888412017167
F1 Score: 0.0009905111054120755

weighted
Precision: 0.03308093930610548
Recall: 0.0024669847080455534
F1 Score: 0.004591557659651667

Mean FNR: 0.9994678111587983
Mean TNR: 0.6293356034110408
Mean FPR: 0

In [12]:
epsilon_values = [0.01, 0.1, 0.2, 0.3]

print("start BA")
for epsilon in epsilon_values:
    filename = f'/home/jovyan/Wustl_iiot/transfer_attack/x_test_adv_BA_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'BA', epsilon)

start BA
Accuracy: 0.999899477702898

macro
Precision: 0.9297211895502187
Recall: 0.9525134250862406
F1 Score: 0.9361010769605054

weighted
Precision: 0.9999216196187077
Recall: 0.999899477702898
F1 Score: 0.999905182190384

Mean FNR: 0.04748657491375938
Mean TNR: 0.9999798326082697
Mean FPR: 2.0167391730416082e-05
Mean TPR: 0.9525134250862406
Accuracy: 0.9998869124157602

macro
Precision: 0.9099160945842868
Recall: 0.9363344180091755
F1 Score: 0.9195799422519716

weighted
Precision: 0.9999074104788865
Recall: 0.9998869124157602
F1 Score: 0.9998932828302103

Mean FNR: 0.06366558199082471
Mean TNR: 0.9999773777315205
Mean FPR: 2.262226847956318e-05
Mean TPR: 0.9363344180091755
Accuracy: 0.9998911008448061

macro
Precision: 0.917142857142857
Recall: 0.9541022084711404
F1 Score: 0.9320111955217912

weighted
Precision: 0.9999139577004567
Recall: 0.9998911008448061
F1 Score: 0.9998984601541222

Mean FNR: 0.04589779152885949
Mean TNR: 0.9999782155647162
Mean FPR: 2.178443528377413e-05
Mean T